# Introduction

The notebook is intended to analyse the weather outside conditions to find correlations with the inside air pollutions.

In [1]:
# Import Standard Libraries
import requests
import xarray as xr

In [2]:
# Notebook's variables
precipitation_data_url = 'https://data.geo.admin.ch/ch.meteoschweiz.klimanormwerte-niederschlag_aktuelle_periode/netcdf/2056/RnormM9120_ch01r.swiss.lv95_000001010000_000012010000.nc'

# Read Data

In [3]:
# Download data
response = requests.get(precipitation_data_url)

In [4]:
# Save data
with open('./../data/RnormM9120_ch01r.swiss.lv95_000001010000_000012010000.nc', 'w') as file:
    file.write(response.text)

In [4]:
# Retrieve data and transform to a CSV
precipitation_data_nc = xr.open_dataset('./../data/RnormM9120_ch01r.swiss.lv95_000001010000_000012010000.nc', decode_times=False)
precipitation_data_pd = precipitation_data_nc.to_dataframe()

In [5]:
precipitation_data_pd.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2131200 entries, (1.0, 0.0, 2474500.0, 1064500.0) to (2.0, 11.0, 2843500.0, 1303500.0)
Data columns (total 5 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   swiss_lv95_coordinates  float64
 1   climatology_bounds      float64
 2   RnormM9120              float32
 3   lon                     float32
 4   lat                     float32
dtypes: float32(3), float64(2)
memory usage: 69.1 MB


In [7]:
precipitation_data_pd.head(20)

swiss_lv95_coordinates  climatology_bounds  \
ncb time E         N                                                       
1.0 0.0  2474500.0 1064500.0                     1.0                 0.0   
                   1065500.0                     1.0                 0.0   
                   1066500.0                     1.0                 0.0   
                   1067500.0                     1.0                 0.0   
                   1068500.0                     1.0                 0.0   
                   1069500.0                     1.0                 0.0   
                   1070500.0                     1.0                 0.0   
                   1071500.0                     1.0                 0.0   
                   1072500.0                     1.0                 0.0   
                   1073500.0                     1.0                 0.0   
                   1074500.0                     1.0                 0.0   
                   1075500.0                     1.0                 0.0   
                   1076500.0                     1.0                 0.0   
                   1077500.0                     1.0                 0.0   
                   1078500.0                     1.0                 0.0   
                   1079500.0                     1.0                 0.0   
                   1080500.0                     1.0                 0.0   
                   1081500.0                     1.0                 0.0   
                   1082500.0                     1.0                 0.0   
                   1083500.0                     1.0                 0.0   

                              RnormM9120       lon        lat  
ncb time E         N                                           
1.0 0.0  2474500.0 1064500.0         NaN  5.826852  45.720608  
                   1065500.0         NaN  5.826589  45.729603  
                   1066500.0         NaN  5.826325  45.738594  
                   1067500.0         NaN  5.826061  45.747589  
                   1068500.0         NaN  5.825797  45.756580  
                   1069500.0         NaN  5.825533  45.765575  
                   1070500.0         NaN  5.825269  45.774567  
                   1071500.0         NaN  5.825005  45.783562  
                   1072500.0         NaN  5.824740  45.792553  
                   1073500.0         NaN  5.824476  45.801548  
                   1074500.0         NaN  5.824212  45.810539  
                   1075500.0         NaN  5.823947  45.819534  
                   1076500.0         NaN  5.823683  45.828526  
                   1077500.0         NaN  5.823418  45.837521  
                   1078500.0         NaN  5.823153  45.846512  
                   1079500.0         NaN  5.822888  45.855507  
                   1080500.0         NaN  5.822623  45.864498  
                   1081500.0         NaN  5.822358  45.873493  
                   1082500.0         NaN  5.822093  45.882484  
                   1083500.0         NaN  5.821828  45.891479